In [ ]:
#import skimage.io

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.colors import LogNorm

import scipy.signal

In [ ]:
from util import plotstyle

In [ ]:
plotstyle.load('screen_dark')
#plotstyle.load('pretty_print')
#plotstyle.load('print')

In [ ]:
from util import bfp

# Calibrate using a Calibration Dataset
## Create Wrapper Object

In [ ]:
calibration_dataset = bfp.SliceDataset()

## Load Image Data

In [ ]:
calibration_dataset.load_image_data( bfp.TIFF_Stack().load_file("../../labdata/TIFF/211111/calibration_test_01.tif") )

In [ ]:
calibration_dataset.img_data_avg.shape

In [ ]:
fig = plt.figure(figsize=(10,5), dpi=100)

axs = fig.add_gridspec(1, 1)

ax = fig.add_subplot(axs[ 0 , 0 ])

im = ax.imshow( np.clip( calibration_dataset.img_data_avg, 0.00003, 1), 
                norm=LogNorm(vmax=1), 
                cmap=plotstyle.cmap('a'))
ax.set_xlabel('$x$ [px]')
ax.set_ylabel('$y$ [px]')
plt.colorbar(im, ax=ax)
ax.set_title("Cumulative sCMOS Image")

plt.tight_layout()
plt.show()

## Generate Mask

In [ ]:
calibration_dataset.gen_mask()

In [ ]:
fig = plt.figure(figsize=(10,5), dpi=100)

axs = fig.add_gridspec(1, 1)

ax = fig.add_subplot(axs[ 0 , 0 ])

im = ax.imshow( calibration_dataset.img_mask, cmap=plotstyle.cmap('bin'))
ax.set_xlabel('$x$ [px]')
ax.set_ylabel('$y$ [px]')
plt.colorbar(im, ax=ax)
ax.set_title("Mask")

plt.tight_layout()
plt.show()

## Generate Regions of Interest from the Mask

In [ ]:
calibration_dataset.gen_rois()

In [ ]:
[ roi.to_string() for roi in calibration_dataset.rois ]

In [ ]:
fig = plt.figure(figsize=(12,10), dpi=100)

axs = fig.add_gridspec( 1, int(np.sum(np.array([ r.width() for r in calibration_dataset.rois ])))+len(calibration_dataset.rois) )

for i in range(len(calibration_dataset.rois)):
    ax = fig.add_subplot(
        axs[ 0 , int(np.sum(np.array([ r.width() for r in calibration_dataset.rois[:i] ])))+i:int(np.sum(np.array([ r.width() for r in calibration_dataset.rois[:i+1] ])))+i ]
    )
    im = ax.imshow( 
        calibration_dataset.rois[i](calibration_dataset.img_data_avg), 
        cmap=plotstyle.cmap('a'), vmin=0, aspect='auto',
        extent=calibration_dataset.rois[i].to_extent()
    )
    if i==0:
        ax.set_ylabel('$y$ [px]')
    else:
        ax.set(yticklabels=[])
    if len(calibration_dataset.rois)//2 == i:
        ax.set_xlabel('$x$ [px]')
    ax.set_title("ROI {i}".format(i=i))

#plt.tight_layout()
plt.show()

## Compute the Calibration Constant

In [ ]:
calibration = calibration_dataset.get_calibration()

In [ ]:
fig = plt.figure(figsize=(12,10), dpi=100)

axs = fig.add_gridspec( 1, int(np.sum(np.array([ r.width() for r in calibration_dataset.rois ])))+len(calibration_dataset.rois) )

for i in range(len(calibration_dataset.rois)):
    ax = fig.add_subplot(
        axs[ 0 , int(np.sum(np.array([ r.width() for r in calibration_dataset.rois[:i] ])))+i:int(np.sum(np.array([ r.width() for r in calibration_dataset.rois[:i+1] ])))+i ]
    )
    EXTENT = calibration_dataset.rois[i].to_extent()
    EXTENT[0] -= calibration_dataset.rois[0].get_xmean()
    EXTENT[1] -= calibration_dataset.rois[0].get_xmean()
    im = ax.imshow( 
        calibration_dataset.rois[i](calibration_dataset.img_data_avg), 
        cmap=plotstyle.cmap('a'), vmin=0, aspect='auto',
        extent=EXTENT
    )
    ax.fill_betweenx( 
        #np.arange(calibration_dataset.roi_mids.shape[1]),
        calibration_dataset.rois[i].Y(),
        #np.convolve( calibration_dataset.roi_mids[i]-calibration_dataset.roi_devs[i], np.array([1,1,1,1,1])/5, mode='same' )+calibration_dataset.rois[i].xmin, 
        #np.convolve( calibration_dataset.roi_mids[i]+calibration_dataset.roi_devs[i], np.array([1,1,1,1,1])/5, mode='same' )+calibration_dataset.rois[i].xmin, 
        calibration_dataset.roi_mids[i] - calibration_dataset.roi_devs[i] - calibration_dataset.rois[0].get_xmean(), 
        calibration_dataset.roi_mids[i] + calibration_dataset.roi_devs[i] - calibration_dataset.rois[0].get_xmean(), 
        color='r', alpha=2*plotstyle.err_alpha() )
    #ax.plot( 
    #    calibration_dataset.roi_mids[i] + calibration_dataset.rois[i].xmin, 
    #    np.arange(calibration_dataset.roi_mids.shape[1]),
    #    color='r', lw=1 )
    ax.plot( 
        [ np.mean(calibration_dataset.roi_mids[i])+calibration_dataset.rois[i].xmin-calibration_dataset.rois[0].get_xmean(), 
          np.mean(calibration_dataset.roi_mids[i])+calibration_dataset.rois[i].xmin-calibration_dataset.rois[0].get_xmean()], 
        [ -0.5, calibration_dataset.roi_mids.shape[1]-0.5 ],
        color=plotstyle.monochrome_fg(), lw=1, ls=':' )
    
    #limits = calibration_dataset.rois[i].to_extent()
    ax.set_xlim( xmin=EXTENT[0], xmax=EXTENT[1] )
    ax.set_ylim( ymin=EXTENT[2], ymax=EXTENT[3] )
    #del limits
    
    if i==0:
        ax.set_ylabel('$y$ [px]')
    else:
        ax.set(yticklabels=[])
    if len(calibration_dataset.rois)//2 == i:
        ax.set_xlabel('$\Delta x$ [px]')
    ax.set_title("ROI {i}".format(i=i))

#del limits
del EXTENT

#plt.tight_layout()
plt.show()

In [ ]:
for roi in calibration_dataset.rois:
    print( roi.to_string() )

In [ ]:
print( calibration.to_dict() )

In [ ]:
print( 
    "{a:.3f} nm/px ± {da:.4f} nm/px\n{b:.3f} px/nm ± {db:.4f} px/nm".format( 
        a=calibration.px_to_lda(), da=calibration.lda_error(),
        b=calibration.lda_to_px(), db=calibration.px_error() 
    ) 
)

In [ ]:
#print("Calibration Done. Press [ENTER] to continue.")

In [ ]:
#input()

## Cleanup

In [ ]:
del calibration_dataset

# Extract a Spectral Correction from one Slice Image

In [ ]:
slice_dataset = bfp.SliceDataset()
#slice_dataset.load_image_data( bfp.TIFF_Stack().load_file("../../labdata/TIFF/211112/scatterer01/lineprofile-12500µm.tif") )

# Calibrate for the Halogen Lamp
slice_dataset.load_image_data( bfp.TIFF_Stack().load_file("../../labdata/TIFF/211112/lamp-spectrum-01.tif") )

# Calibrate for the LED
#slice_dataset.load_image_data( bfp.TIFF_Stack().load_file("../../labdata/TIFF/211111/test.tif") )
#slice_dataset.load_image_data( bfp.TIFF_Stack().load_file("../../labdata/TIFF/211111/test02.tif") )



In [ ]:
slice_dataset.gen_mask()


In [ ]:
fig = plt.figure(figsize=(11,11), dpi=100)

axs = fig.add_gridspec(2, 1)

ax = fig.add_subplot(axs[ 0 , 0 ])

im = ax.imshow( np.clip( slice_dataset.img_data_avg, 0.00003, 1), 
                norm=LogNorm(vmax=1), 
                cmap=plotstyle.cmap('a'))
ax.set_xlabel('$x$ [px]')
ax.set_ylabel('$y$ [px]')
plt.colorbar(im, ax=ax)
ax.set_title("Cumulative sCMOS Image")

ax = fig.add_subplot(axs[ 1 , 0 ])

im = ax.imshow( slice_dataset.img_mask, cmap=plotstyle.cmap('bin'))
ax.set_xlabel('$x$ [px]')
ax.set_ylabel('$y$ [px]')
plt.colorbar(im, ax=ax)
ax.set_title("Mask")

plt.tight_layout()
plt.show()

In [ ]:
slice_dataset.gen_rois()

In [ ]:
slice_dataset.map_rois()

In [ ]:
slice_dataset.SMEAR.xmin = int(slice_dataset.SLICE.xmid()+slice_dataset.SLICE.width())

In [ ]:
slice_dataset.correct_top_bottom_bg()

In [ ]:
fig = plt.figure(figsize=(12,10), dpi=100)

axs = fig.add_gridspec( 1, int( slice_dataset.SLICE.width() + slice_dataset.SMEAR.width() ) + 10 )

EXTENT = slice_dataset.SLICE.to_extent()
EXTENT[0] -= slice_dataset.SLICE.xmid()
EXTENT[1] -= slice_dataset.SLICE.xmid()

ax = fig.add_subplot( axs[ 0 , :int( slice_dataset.SLICE.width() ) ] )
im = ax.imshow( 
    slice_dataset.SLICE(slice_dataset.img_data_avg), 
    #norm=LogNorm(vmax=1, vmin=0.0001), 
    vmin=0,
    cmap=plotstyle.cmap('a'), aspect='auto',
    extent=EXTENT
)
ax.set_ylabel('$y$ [px]')
ax.set_xlabel('$\Delta x$ [px]')
ax.set_title("SLICE")

ax.grid()



EXTENT = slice_dataset.SMEAR.to_extent()
EXTENT[0] -= slice_dataset.x0_px
EXTENT[1] -= slice_dataset.x0_px
EXTENT[0] *= calibration.px_to_lda()
EXTENT[1] *= calibration.px_to_lda()

ax = fig.add_subplot( axs[ 0 , int( slice_dataset.SLICE.width() )+10: ] )
im = ax.imshow( 
    slice_dataset.SMEAR(slice_dataset.img_data_avg), 
    #norm=LogNorm(vmax=1, vmin=0.0001), 
    vmin=0,
    cmap=plotstyle.cmap('a'), aspect='auto',
    extent=EXTENT
)
ax.set(yticklabels=[])
ax.set_xlabel('$\lambda_\mathrm{corresp.}$ [nm]')
ax.set_title("SMEAR")

ax.grid()
    
del EXTENT
    
#plt.tight_layout()
plt.show()

In [ ]:
#lineweights = np.sum( slice_dataset.SMEAR(slice_dataset.img_data_avg), axis=1 )
#lineweights /= np.sum(lineweights)

#mean_spec = np.mean( slice_dataset.SMEAR(slice_dataset.img_data_avg), axis=0 )
#mean_spec /= np.max(mean_spec)


In [ ]:
fig = plt.figure(figsize=(12,9), dpi=100)

axs = fig.add_gridspec( 1, 1 )

ax = fig.add_subplot( axs[ 0 , : ] )

BINNING = 4
for i in np.arange( 0, slice_dataset.SMEAR(slice_dataset.img_data_avg).shape[0], BINNING ):
    ax.plot(
        calibration.px_to_lda( slice_dataset.SMEAR.X()-slice_dataset.x0_px ),
        np.mean( slice_dataset.SMEAR(slice_dataset.img_data_avg)[i:i+BINNING], axis=0)/np.max(np.mean( slice_dataset.SMEAR(slice_dataset.img_data_avg)[i:i+BINNING], axis=0)),
        color=plotstyle.c(0), #monochrome_fg(),
        lw=1, ls='-', alpha=np.max(np.mean( slice_dataset.SMEAR(slice_dataset.img_data_avg)[i:i+BINNING], axis=0))
    )
del BINNING

ax.plot(
    calibration.px_to_lda( slice_dataset.SMEAR.X()-slice_dataset.x0_px ),
    np.mean( slice_dataset.SMEAR(slice_dataset.img_data_avg), axis=0 )/np.max(np.mean( slice_dataset.SMEAR(slice_dataset.img_data_avg), axis=0 )),
    color=plotstyle.monochrome_fg()    
)

#ax.set(yticklabels=[])
ax.set_ylabel('Rel. Spectral Efficiency [a.u.]')
ax.set_xlabel('$\lambda_\mathrm{corresp.}$ [nm]')
ax.set_title("Spectral Correction")

ax.grid()
    

    
plt.tight_layout()
plt.show()

## Generate an Interpolation Function for the Spectral Correction

In [ ]:
from scipy import interpolate

In [ ]:
LDA = calibration.px_to_lda( slice_dataset.SMEAR.X()-slice_dataset.x0_px )
VAL = np.mean( slice_dataset.SMEAR(slice_dataset.img_data_avg), axis=0 )/np.max(np.mean( slice_dataset.SMEAR(slice_dataset.img_data_avg), axis=0 ))

In [ ]:
LDA = np.append( LDA, 1000.0 )
VAL = np.append( VAL, 0.0 )

In [ ]:
correction = scipy.interpolate.interp1d( LDA, VAL, kind=2 )
correction_valid_range = (np.min(LDA), np.max(LDA))


In [ ]:
correction_valid_range

## Cleanup

In [ ]:
del LDA
del VAL

del slice_dataset

# Building a full 3D BFP Dataset

## Filtering Functions to convert Spectra to Colour

In [ ]:
#from util import colour_system as cs
from util import colour

cs = colour.cs_hdtv


In [ ]:
LDA = np.linspace( correction_valid_range[0], correction_valid_range[1], 400 )

In [ ]:
RGB_map = [ cs.spec_to_rgb([lda], [1]) for lda in LDA ]
#for i in range( len(RGB) ):
#    RGB[i] /= np.sum(RGB[i])
RGB_map = np.array( [RGB_map] )


In [ ]:
fig = plt.figure(figsize=(12,6), dpi=100)

axs = fig.add_gridspec( 4, 1 )


ax = fig.add_subplot( axs[ :-1 , : ] )

ax.fill_between( LDA, RGB_map[0,:,0], alpha=0.2 )
ax.plot( LDA, RGB_map[0,:,0], label='Red Channel' )

ax.fill_between( LDA, RGB_map[0,:,1], alpha=0.2 )
ax.plot( LDA, RGB_map[0,:,1], label='Green Channel' )

ax.fill_between( LDA, RGB_map[0,:,2], alpha=0.2 )
ax.plot( LDA, RGB_map[0,:,2], label='Blue Channel' )

ax.set_xlim( xmin=np.min(LDA), xmax=np.max(LDA) )
ax.set_ylabel('Activation [a.u.]')
ax.set_xlabel('$\lambda$ [nm]')
ax.set_title("RGB Components")

ax.legend()
ax.grid()


ax = fig.add_subplot( axs[ -1 , : ] )

ax.imshow(RGB_map, extent=( np.min(LDA), np.max(LDA), 1, 0 ), aspect='auto')

#ax.set_ylabel('$y$ [px]')
ax.set(yticklabels=[])
ax.set_xlabel('$\lambda$ [nm]')
ax.set_title("Colour Representation")

ax.grid()


plt.tight_layout()
plt.show()

### Test on the Correction Spectrum

In [ ]:

def planck(wav, T=6000):
    pi = np.pi
    h = 6.626e-34
    c = 3.0e+8
    k = 1.38e-23
    
    a = 2.0*h*c**2
    b = h*c/(1e-9*wav*k*T)
    intensity = a/ ( (1e-9*wav**5) * (np.exp(b) - 1.0) )
    return intensity

def flat(lda):
    return (0.0*lda)+1.0


In [ ]:
test_spec = correction(LDA)
#test_spec = flat(LDA)
#test_spec = planck(LDA, 6000)

test_spec /= np.max(test_spec)


In [ ]:
RGB = cs.spec_to_rgb(LDA, test_spec)
RGB.shape

In [ ]:
for i in [0,1,2]:
    RGB_map[0,:,i] *= test_spec

In [ ]:
fig = plt.figure(figsize=(12,6), dpi=100)

axs = fig.add_gridspec( 4, 1 )


ax = fig.add_subplot( axs[ :-1 , : ] )

ax.fill_between( LDA, RGB_map[0,:,0], alpha=0.2 )
ax.plot(         LDA, RGB_map[0,:,0] )

ax.fill_between( LDA, RGB_map[0,:,1], alpha=0.2 )
ax.plot(         LDA, RGB_map[0,:,1] )

ax.fill_between( LDA, RGB_map[0,:,2], alpha=0.2 )
ax.plot(         LDA, RGB_map[0,:,2] )

ax.fill_between( LDA, test_spec, (0.0*LDA)+1.0, color=plotstyle.monochrome_fg(), alpha=plotstyle.err_alpha() )
#ax.fill_between( LDA, correction(LDA), color=plotstyle.monochrome_fg(), alpha=plotstyle.err_alpha() )
ax.plot( LDA, test_spec, color=plotstyle.monochrome_fg(), ls=':' )


ax.set_xlim( xmin=np.min(LDA), xmax=np.max(LDA) )
ax.set_ylabel('Activation [a.u.]')
ax.set_xlabel('$\lambda$ [nm]')
ax.set_title("Filtering")

ax.grid()



ax = fig.add_subplot( axs[ -1 , : ] )

ax.imshow(RGB_map, extent=( np.min(LDA), np.max(LDA), 1, 0 ), aspect='auto')

#ax.set_ylabel('$y$ [px]')
ax.set(yticklabels=[])
ax.set_xlabel('$\lambda$ [nm]')
ax.set_title("Colour Representation")


plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(8,1), dpi=100)

axs = fig.add_gridspec( 1, 1 )

ax = fig.add_subplot( axs[ : , : ] )

ax.imshow(RGB[np.newaxis,np.newaxis], extent=( 0, 1, 1, 0 ), aspect='auto')

ax.axis('off')
ax.set_title("Percieved Colour")

plt.tight_layout()
plt.show()

In [ ]:
del LDA

## Conversion of Offset on the Slit to Pixel Offset

In [ ]:
M2 = 2.0 # magnification from slit to sensor
px_size = 6.5 # micrometres
um_to_px_offset = M2/px_size
print( "{:.3f} px/µm".format(um_to_px_offset) )
print( "Offset @ 50 µm: {:.3f} px".format(50*um_to_px_offset) )

## Load each Slice

In [ ]:
shifts_um = np.arange( 11850, 13150+50, 50 )
files = [ "../../labdata/TIFF/211112/scatterer01/lineprofile-{:}µm.tif".format(shift) for shift in shifts_um ]
files

In [ ]:
slices = [ bfp.SliceDataset().load_image_data( bfp.TIFF_Stack().load_file( FILE ) ) for FILE in files ]


In [ ]:
for sl in slices:
    sl.gen_mask()
    sl.gen_rois()
    sl.map_rois()
    sl.correct_top_bottom_bg()


In [ ]:
#for sl in slices:
#    sl.SMEAR.xmin = int(sl.SLICE.xmid()+sl.SLICE.width())
    

In [ ]:
fig = plt.figure(figsize=(12,6*len(slices)), dpi=100)

axs = fig.add_gridspec( len(slices), 10 )

i=0
for sl in slices:
    EXTENT = sl.SLICE.to_extent()
    EXTENT[0] -= sl.SLICE.xmid()
    EXTENT[1] -= sl.SLICE.xmid()

    ax = fig.add_subplot( axs[ i , :1 ] )
    im = ax.imshow( 
        sl.SLICE(sl.img_data_avg), 
        #norm=LogNorm(vmax=1, vmin=0.0001), 
        vmin=0,
        cmap=plotstyle.cmap('a'), aspect='auto',
        extent=EXTENT
    )
    ax.set_ylabel('$y$ [px]')
    ax.set_xlabel('$\Delta x$ [px]')
    ax.set_title("SLICE")

    ax.grid()



    EXTENT = sl.SMEAR.to_extent()
    EXTENT[0] -= sl.x0_px
    EXTENT[1] -= sl.x0_px
    EXTENT[0] *= calibration.px_to_lda()
    EXTENT[1] *= calibration.px_to_lda()

    ax = fig.add_subplot( axs[ i , 1: ] )
    im = ax.imshow( 
        sl.SMEAR(sl.img_data_avg), 
        #norm=LogNorm(vmax=1, vmin=0.0001), 
        vmin=0,
        cmap=plotstyle.cmap('a'), aspect='auto',
        extent=EXTENT
    )
    ax.set(yticklabels=[])
    ax.set_xlabel('$\lambda_\mathrm{corresp.}$ [nm]')
    ax.set_title("SMEAR")

    ax.grid()

    del EXTENT
    i += 1
del i
    
plt.tight_layout()
plt.show()

In [ ]:
fig.savefig("Slice Stack.pdf", bbox_inches='tight', dpi=100)
#fig.savefig("Assembled TruColour Images (v4).png", bbox_inches='tight', dpi=400)


## Prepare Storage for combined Dataset

In [ ]:
layers = shifts_um.shape[0]
im_height = 1024
shifts_px = np.round( um_to_px_offset*( shifts_um - np.min(shifts_um) ) ).astype(int)
#shifts_px = np.round( um_to_px_offset*( -shifts_um + np.min(shifts_um) ) ).astype(int)
#shifts_px = np.round( -um_to_px_offset*( shifts_um - np.max(shifts_um) ) ).astype(int)
#shifts_px *= -1
#shifts_px -= np.min(shifts_px)
# required width of the image stack, with some margin
im_width = np.max(shifts_px)-np.min(shifts_px)+5*np.max(np.diff(shifts_px))
shifts_px += np.max(np.diff(shifts_px))
#shifts_px -= np.max(np.diff(shifts_px))

images = np.zeros( (layers, im_height, im_width) )


In [ ]:
shifts_px = np.flipud(shifts_px) # TODO: do this properly, somehow through the offset factor

In [ ]:
#images.shape
shifts_px
#im_width

## Assemble the proper 0th order image

In [ ]:
peak_norm = np.max( np.array( [ sl.NORM_FAC for sl in slices ] ) )
for sl in slices:
    sl.renormalize(peak_norm)

In [ ]:
#for i in range(layers):
#    print(slices[i].img_data_avg[:,slices[i].SLICE.xslice()].shape, " -> ", images[i,:,shifts_px[i]:shifts_px[i]+slices[i].SLICE.width()].shape )

In [ ]:
for i in range(layers):
    images[i,:,shifts_px[i]:shifts_px[i]+slices[i].SLICE.width()] = np.copy( slices[i].img_data_avg[:,slices[i].SLICE.xslice()] ) # normalization factor, have to keep track of inside SliceDataset

In [ ]:
fig = plt.figure(figsize=(12,12), dpi=100)

axs = fig.add_gridspec( 1, 10 )

ax = fig.add_subplot( axs[ : , : ] )
im = ax.imshow( 
    np.sum(images, axis=0), 
    #images[5], 
    #norm=LogNorm(vmax=1, vmin=0.0001), 
    vmin=0,
    cmap=plotstyle.cmap('a'), #aspect='auto',
    #extent=EXTENT
)
ax.set_ylabel('$y$ [px]')
ax.set_xlabel('$x$ [px]')
ax.set_title("Re-assembled Image")

ax.grid()
    
plt.tight_layout()
plt.show()

In [ ]:
fig.savefig("Reassembled 0o Image.pdf", bbox_inches='tight', dpi=100)


## Compute Spectra for each Image Point

In [ ]:
#for sl in slices:
#    print(sl.SMEAR.width())
#np.argmax( np.array( [sl.SMEAR.width() for sl in slices] ) )

In [ ]:
# Index of the slice dataset whose minimum determined wavelength 
i_min_displace = np.argmin( np.array([ sl.SMEAR.xmin - sl.SLICE.xmid() for sl in slices]) )
i_max_displace = np.argmax( np.array([ sl.SMEAR.xmax - sl.SLICE.xmid() for sl in slices]) ) 
# Could have gotten the same result with just the argmin & argmax of SMEAR.width(), but this kinda (maybe) makes it more clear what I'm trying to do...

displace_min = int(slices[i_min_displace].SMEAR.xmin) - int(slices[i_min_displace].SLICE.xmid())
displace_max = int(slices[i_max_displace].SMEAR.xmax) - int(slices[i_max_displace].SLICE.xmid())

displacements = np.arange( displace_min, displace_max+1, 1 )

( np.min(displacements), np.max(displacements) )

#(i_min_displace, i_max_displace)

#del i


In [ ]:

LDA = calibration.px_to_lda(displacements)
( np.min(LDA), np.max(LDA) )


In [ ]:
LDA.shape

In [ ]:
y_binning = 16
y_indices = np.arange(0,im_height,y_binning)
binned_height = y_indices.shape[0]

#binned_images = np.zeros( (layers, binned_height, im_width) )




In [ ]:
spectra = np.zeros( (im_height, im_width, LDA.shape[0]) )

for y in range(im_height):
    for x in range(im_width):
        layer_weights = np.copy( images[:,y,x] )
        #print( np.nonzero(layer_weights) )
        #if np.sum(layer_weights) != 0.0:
        #    layer_weights /= np.sum(layer_weights)
        for L in range(layer_weights.shape[0]):
            #print(L)
            #spec = np.zeros( spectra.shape[2] )
            #start_idx = int(slices[L].SMEAR.xmin  ) - int(slices[L].SLICE.xmid())
            #stop_idx  = int(slices[L].SMEAR.xmax+1) - int(slices[L].SLICE.xmid())
            spectra[y,x,:] += layer_weights[L]*slices[L].img_data_avg[y, displace_min+int(slices[L].SLICE.xmid()):displace_max+int(slices[L].SLICE.xmid())+1]
        if np.sum(spectra[y,x]) > 0:
            spectra[y,x] /= np.sum( np.square(spectra[y,x]) )

## Generate (hopefully) true colour Image

In [ ]:
image_RGB = np.zeros( (im_height, im_width, 4) )

for y in range(im_height):
    for x in range(im_width):
        #image_RGB[y,x] = np.sum(images[:,y,x])*cs.spec_to_rgb(LDA, spectra[y,x])
        image_RGB[y,x,0:3] = cs.spec_to_rgb(LDA, spectra[y,x])

image_RGB[:,:,3] += 1.0


This is to have the intensity kinda logarithmic, so that you can see more than just the peaking spot, but also clipped such that the background is ignored

In [ ]:
alphamap = np.sum(images, axis=0)
alphamap /= np.max(alphamap)
alphamap = np.clip(alphamap, 0.0, 1.0)
#alphamap = np.log10( 100*np.clip(alphamap, 0.01, 1.0) )/2
image_RGB[:,:,3] = alphamap

## Generate another one, with the Spectral Correction applied

In [ ]:
#for y in range(im_height):
#    for x in range(im_width):
#        spectra[y,x] /= correction(LDA)

In [ ]:
corr_image_RGB = np.zeros( (im_height, im_width, 4) )

for y in range(im_height):
    for x in range(im_width):
        #image_RGB[y,x] = np.sum(images[:,y,x])*cs.spec_to_rgb(LDA, spectra[y,x])
        corr_image_RGB[y,x,0:3] = cs.spec_to_rgb(LDA, spectra[y,x]/correction(LDA))

corr_image_RGB[:,:,3] += 1.0


In [ ]:
alphamap = np.sum(images, axis=0)
alphamap /= np.max(alphamap)
alphamap = np.log10( 100*np.clip(alphamap, 0.01, 1.0) )/2
corr_image_RGB[:,:,3] = alphamap

## Generate a third one, whitebalance it

In [ ]:
wb_image_RGB = np.zeros( (im_height, im_width, 4) )

for y in range(im_height):
    for x in range(im_width):
        #image_RGB[y,x] = np.sum(images[:,y,x])*cs.spec_to_rgb(LDA, spectra[y,x])
        wb_image_RGB[y,x,0:3] = cs.spec_to_rgb(LDA, spectra[y,x]/correction(LDA)*planck(LDA, 6000))

wb_image_RGB[:,:,3] += 1.0


In [ ]:
alphamap = np.sum(images, axis=0)
alphamap /= np.max(alphamap)
alphamap = np.log10( 100*np.clip(alphamap, 0.01, 1.0) )/2
wb_image_RGB[:,:,3] = alphamap

In [ ]:
#wb_red_RGB = np.copy(wb_image_RGB)
#wb_green_RGB = np.copy(wb_image_RGB)
#wb_blue_RGB = np.copy(wb_image_RGB)

#wb_red_RGB[:,:,1] *= 0.0
#wb_red_RGB[:,:,2] *= 0.0
#wb_green_RGB[:,:,0] *= 0.0
#wb_green_RGB[:,:,2] *= 0.0
#wb_blue_RGB[:,:,0] *= 0.0
#wb_blue_RGB[:,:,1] *= 0.0


## Generate one with the brightness scaled with contrast

In [ ]:
contrast_image_RGB = np.zeros( (im_height, im_width, 4) )

for y in range(im_height):
    for x in range(im_width):
        #image_RGB[y,x] = np.sum(images[:,y,x])*cs.spec_to_rgb(LDA, spectra[y,x])
        contrast_image_RGB[y,x,0:3] = cs.spec_to_rgb(LDA, spectra[y,x]/correction(LDA)*planck(LDA, 6000))

contrast_image_RGB[:,:,3] += 1.0


This should filter out everything that lies outside the main ROI and then scale the alpha (brightness) with the $\mathcal{L}^2$-Norm of the difference of the spectrum at the image point and the correction (background) spectrum.

In [ ]:
alphamap = np.sum(images, axis=0)
#alphamap /= np.max(alphamap)
#alphamap = np.log10( 100*np.clip(alphamap, 0.01, 1.0) )/2
alphamap = np.where( alphamap>0.01, 1.0, 0.0 )

for y in range(im_height):
    for x in range(im_width):
        #if not np.any(np.isnan(spectra[y,x])):
        if np.sum(spectra[y,x]) != 0:
            alphamap[y,x] *= np.sum( np.square( spectra[y,x]/np.sum(spectra[y,x]) - correction(LDA)/np.sum(correction(LDA)) ) )
        

#contrast_image_RGB[:,:,3] /= np.max( contrast_image_RGB[:,:,3] )

In [ ]:
#alphamap /= np.max(alphamap)
#alphamap = np.log10(alphamap+0.01)

In [ ]:
contrast_image_RGB[:,:,3] = alphamap/np.max(alphamap)

## Plot

In [ ]:
fig = plt.figure(figsize=(12,12), dpi=100)

axs = fig.add_gridspec( 2, 2 )


ax = fig.add_subplot( axs[ 0 , 0 ] )
ax.set_facecolor((0.0,0.0,0.0))
im = ax.imshow( 
    image_RGB
    #extent=EXTENT
)
ax.set_ylim( ymin=750, ymax=250 )
ax.set_ylabel('$y$ [px]')
ax.set_xlabel('$x$ [px]')
ax.set_title("Assembled Image\nReconstructed Colour")

#ax.grid()


ax = fig.add_subplot( axs[ 0 , 1 ] )
ax.set_facecolor((0.0,0.0,0.0))
im = ax.imshow( 
    corr_image_RGB
    #extent=EXTENT
)
ax.set_ylim( ymin=750, ymax=250 )
ax.set_ylabel('$y$ [px]')
ax.set_xlabel('$x$ [px]')
ax.set_title("Assembled Image\nSpectrally Corrected Colour, Logarithmic Brightness")

#ax.grid()


ax = fig.add_subplot( axs[ 1 , 0 ] )
ax.set_facecolor((0.0,0.0,0.0))
im = ax.imshow( 
    wb_image_RGB
    #extent=EXTENT
)
ax.set_ylim( ymin=750, ymax=250 )
ax.set_ylabel('$y$ [px]')
ax.set_xlabel('$x$ [px]')
ax.set_title("Assembled Image\nWhite Balanced @ 6000 K")

#ax.grid()


ax = fig.add_subplot( axs[ 1 , 1 ] )
ax.set_facecolor((0.0,0.0,0.0))
im = ax.imshow( 
    contrast_image_RGB#[:,:,3]
    #extent=EXTENT
)
ax.set_ylim( ymin=750, ymax=250 )
ax.set_ylabel('$y$ [px]')
ax.set_xlabel('$x$ [px]')
ax.set_title("Assembled Image\nSpectral Deviation Map")

#ax.grid()


plt.tight_layout()
plt.show()

In [ ]:
fig.savefig("Assembled TruColour Images (v4).pdf", bbox_inches='tight', dpi=100)
fig.savefig("Assembled TruColour Images (v4).png", bbox_inches='tight', dpi=400)
